In [ ]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from datetime import datetime
import requests
import sys
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import random
import operator
import re

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#/Users/leopoldo.alarcon/Documents/scriptsRRSS/chromedriver 2
#https://pbpython.com/pandas-html-table.html

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [ ]:
browser.get('https://www.liverpool.com.mx/tienda/pdp/Set-Peppa-Pig-Crea-y-Dise%C3%B1a-Play-Doh/1109740311#ratings')

In [ ]:
#pages = range(86,88,1)
data = pd.DataFrame(columns=['FECHA', 'RATE', 'TITULO', 'REVIEW', 'GENERO','PROYECTO','PRODUCTO'])

In [ ]:
def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    # Nos pasa el textoo a minúsculas.
    txt = re.sub(' +',' ', txt)
    # Limpia nuestro texto.
    return txt.strip().split()
    # Nos regresa el texto con las delimitaciones especificadas.

def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

gender_list = pd.read_csv('nombres.csv')
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')

In [ ]:
elem = browser.find_element_by_xpath("//*")
soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
reviews = soup.find_all("div",'TTreview')
print(len(reviews))
for rev in reviews:
    fecha = rev.find_all("div",'TTrevCol3')[0].get_text().split('\n')[1]
    autor = rev.find_all("div",'TTrevCol3')[0].get_text().split('\n')[4]
    rate = rev.find_all("div",'TThiddenForADA')[0].get_text()[-6]
    title = rev.find_all("div",'TTreviewTitle')[0].text.strip('\n')
    comen = rev.find_all("div",'TTreviewBody')[0].text.strip('\n').strip(' ')
    gen = get_gender2(autor)
    data = data.append({'FECHA': fecha,'AUTOR':autor,'RATE':rate,'TITULO':title,
                'REVIEW':comen,'GENERO':gen,
                'PROYECTO':'HASBRO','PRODUCTO':'Set Peppa Pig Crea y Diseña Play-Doh'},ignore_index=True)

8


In [ ]:
data

,FECHA,RATE,TITULO,REVIEW,GENERO,PROYECTO,PRODUCTO,AUTOR
0,22 de octubre de 2021,5,,Super la entrega muy a tiempo,f,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,Erika M
1,30 de octubre de 2021,5,,,a,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,Betzabe G
2,26 de marzo de 2022,5,Buena compra,Muy lindas,a,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,Milendi A
3,5 de diciembre de 2021,5,,,f,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,CINTHIA NAVIL F
4,3 de diciembre de 2021,5,,,m,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,Juan Fernando C
5,6 de enero de 2022,5,,,f,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,viridiana c
6,10 de diciembre de 2021,5,Todo viene tal cual,"Genial, a mis hijos les encantó",f,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,Iris C
7,22 de diciembre de 2021,5,,,m,HASBRO,Set Peppa Pig Crea y Diseña Play-Doh,JULIO CESAR O


In [ ]:
data.groupby(['RATE']).count()

,FECHA,TITULO,REVIEW,GENERO,PROYECTO,PRODUCTO,AUTOR
RATE,,,,,,,
5,7,7,7,7,7,7,7


In [ ]:
export_csv = data.to_csv ("PLayDohL9.csv", index = None, header=True , encoding='UTF-8')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=81cf9105-8cbb-4eb6-9fd8-b1c5776e8dac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>